Импортируем нужные для работы библиотеки и подгрузим наш датасет

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from colorama import Fore, Back, Style
# from random import choice

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

data = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
data.head(3)

Посмотрим как устроен наш датасет и какие данные в нем имеются. Воспользуемся для начала функцией ProfileReport() из пакета pandas_profiling

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(data)
profile.to_notebook_iframe()

In [ ]:
y = data['output']
X = data.drop('output', axis=is=1)

In [ ]:
print(Fore.BLUE + f'Датасет состоит из {data.shape[0]} строк и {data.shape[1]} столбцов')

Данные в столбцах имеют следующее значение:

* **age** - возраст;
* **sex** - пол (0 - женщины, 1 -  мужчины);
* **cp** - боли в сердце;
* **trtbps** - значения артериального давления (АД) измерянного в покое;
* **chol** - уровень холестерина (мг/дл);
* **fbs** - значения глюкозо-толерантного теста (если более 120 мг/дл, тогда 1; если меньше 120 мг/дл, тогда 0);
* **restecg** - ЭКГ записанная в покое: 0 - норма, 1 - подъем сегмента ST, 2 - гипертрофия левого желудочка;
* **thalachh** - максимально достигнутая частота сердечных сокращений (ЧСС);
* **exng** - сердечные боли, вызванные физической нагрузкой;
* **oldpeak** - предыдущие пики(?);
* **slp** - наклон
* **caa** - количество основных венозных сосудов
* **thall** - ядерный стресс-тест (с использованием таллия)
* **output** - таргетная переменная (развитие сердечного приступа - 1, отсутствие сердечного приступа - 0)

Опишем поподробнее статистические величины по интересующим нас столбцам:

In [ ]:
def stats_of_column(column: str):
#     global data 
#     data = data[column]
    mean = np.mean(data[column])
    std = np.std(data[column])
    variance = np.var(data[column])
    qrt_25 = np.quantile(data[column], 0.25)
    qrt_50 = np.quantile(data[column], 0.5)
    qrt_75 = np.quantile(data[column], 0.75)
    name = column.capitalize()
    
    color = ['RED', 'BLUE', 'GREEN']
    print(Fore.BLUE + f'Данные в столбце "{name}" имеют следующие характеристики: ')
    print('-'*55)
    print(Style.RESET_ALL + f'1. Среднее значение - {round(mean, 2)}')
    print(f'2. СКО - {round(std, 2)}')
#     print(f'3. Дисперсия - {round(variance, 2)}')
    print(f'3. 25%-квантиль - {round(qrt_25, 2)}')
    print(f'4. 50%-квантиль - {round(qrt_50, 2)}')
    print(f'5. 75%-квантиль - {round(qrt_75, 2)}')
    print()

stats_of_column('age')
stats_of_column('chol')
stats_of_column('trtbps')

Такую же информацию можно получить воспользовавшись встроенной в **pandas** функцией *describe()*.  Для удобства можно вызвать дополнительную функцию *transpose()* чтобы изменить ориентацию таблицы:

In [ ]:
data.describe().transpose()

Как видно из таблицы выше, в датасете все данные полные: в каждом столбце представлено 303 записи. Но мы также можем проверить данные на наличие пустых строк немного другим способом:

In [ ]:
data.isnull().values.any()

Либо воспользовавшись простеньким итератором по столбцам. Примерно такого вида:

In [ ]:
for col in data.columns:
    pct_missing = data[col].isnull().sum()
    print(f'{col} - {pct_missing :.1%}')

## Визуализация данных и EDA

Проведем некоторую визуализацию данных в ходе EDA

In [ ]:
plt.figure(figsize=(18, 10))
plt.style.use("ggplot")
sns.countplot(x=data["age"]);
plt.title("Распределение по возрастам", fontsize=20)
plt.xlabel("ВОЗРАСТ", fontsize=20)
plt.ylabel("КОЛИЧЕСТВО", fontsize=20)
plt.show()

Наибольшую группу лиц в представленном наборе данных составляют люди в возрасте 54 года, а также 57-58 лет. А теперь давайте построим простейший boxplot и посмотрим как представлены значения возраста c разбивкой по полу:

In [ ]:
# sex = data["sex"].value_counts().reset_index()
# px.pie(sex,names="index",values="sex")

# s0 = data[data['sex'] == 0]
# s1 = data[data['sex'] == 1]
# sex = [s0.age,s1.age]
# lab0 = 'Мужчины'
# plt.figure(figsize=(18, 10))
# plt.style.use("ggplot")
# plt.boxplot(sex, showmeans=True, showfliers=False, notch=False)
# plt.show()

In [ ]:
# s0 = data[data['sex'] == 0]
# s1 = data[data['sex'] == 1]
# sex = [s0.age,s1.age]

plt.figure(figsize=(12, 8))
plt.style.use("ggplot")

sns.boxplot(x='sex', y='age', data=data);
sns.swarmplot(x='sex', y='age', data=data, color='.25',)
plt.title("Распределение по половозрастным группам", fontsize=20)
# plt.xlabel("Пол", fontsize=20)
# plt.ylabel("КОЛИЧЕСТВО", fontsize=20)
plt.show()

Как видно из корреляционной матрицы ниже, достаточно значимую прямую корреляцию с исходом заболевания (сердечным приступом) имеют следующие показатели:
1. Боли в сердце (cp);
2. Максимальная достигнутая частота сердечных сокращений (thalachh).

Также можно заметить любопытный факт: имеется обратная зависимость между ЧСС и возрастом, что возможно объясняется снижением компенсаторных возможностей (в том числе и со стороных сердечно-сосудистой системы) с увеличением возраста.

Второе обстоятельство заслуживающее внимание - это отсутствие как-либо значимой зависимости между холестерином и наступлением сердечного приступа. Другими словами: в незасимости от того какой уровень холестерина определяется у человека, у него может наступить или не наступить сердечный приступ практически в равной вероятности!

In [ ]:
data['output'].corr(data['chol'])

In [ ]:
data['output'].corr(data['cp'])

In [ ]:
plt.figure(figsize=(20, 17))
matrix = np.triu(data.corr())
sns.heatmap(data.corr(), annot=True,
            linewidth=.8, mask=matrix, cmap="rocket");

На мой взгляд любопытным представляется распределение некоторых признаков в нашей выборке в целом, а также в разрезе половых групп. Для этих целей приведу несколько графиков

In [ ]:
plt.style.use("ggplot");
sns.displot(data=data, x="thalachh", hue='sex', kde=True, multiple='stack');
plt.title("Максимальная ЧСС",fontsize=18, pad=25);
plt.xlabel("ЧАСТОТА СЕРДЕЧНЫХ СОКРАЩЕНИЙ",fontsize=20);
plt.ylabel("КОЛИЧЕСТВО НАБЛЮДЕНИЙ",fontsize=20);
plt.show();

In [ ]:
plt.style.use("ggplot");
sns.countplot(data=data, x="cp", hue='sex', );
plt.title("Характер сердечных болей",fontsize=18, pad=25);
plt.xlabel("ТИПЫ БОЛИ",fontsize=20);
plt.ylabel("КОЛИЧЕСТВО",fontsize=20);
plt.show();

Как видно из диаграммы выше, чаще всего у мужчин (sex = 1) возникают типичные ангинозные боли, в то время как у женщин распределение между неангинозным характером болей (cp = 2) и типичными ангинозными болям (cp = 0) наблюдается примерно поровну

Продолжая анализировать данные, построим график зависимости характера боли и сердечного приступа:

In [ ]:
plt.style.use("ggplot");
sns.countplot(data=data, x="cp", hue='output');
plt.title("Развитие сердечного приступа\nв зависимости от характера боли",fontsize=18, pad=25);
plt.xlabel("ТИП БОЛИ",fontsize=20);
plt.ylabel("КОЛИЧЕСТВО",fontsize=20);
plt.show();

Как видно из диаграммы выше, чаще всего приступ бывает индуцирован неангинозным характером болей (cp = 2)

Ниже приведены распределения в выборке значений артериального давления и холестерина

In [ ]:
plt.style.use("ggplot");
sns.displot(data=data, x="trtbps", hue='sex', kde=True, multiple='stack');
plt.title("Распределение показателей АД",fontsize=18, pad=25);
plt.xlabel("АРТЕРИАЛЬНОЕ ДАВЛЕНИЕ",fontsize=20)
plt.ylabel("КОЛИЧЕСТВО",fontsize=20);
plt.show();

In [ ]:
plt.style.use("ggplot")
sns.set_color_codes()
sns.displot(data, x="chol", hue='sex', kde=True, multiple='stack')
plt.title("Распределение показателей холестерина", fontsize=18, pad=25)
plt.xlabel("ХОЛЕСТЕРИН", fontsize=20)
plt.ylabel("КОЛИЧЕСТВО", fontsize=20)
plt.show()

Как можно увидеть из приведенных диаграмм, стандартному распределению больше следует распределение холестерина в выборке, однако здесь имеется ряд выбросов в районе значений 400 и выше

## Логистическая регрессия

Теперь займемся определением коэффициента детерминации в нашем датасете, но для начала подготовим данные для проведения логистической регрессии. Для этого сперва определим features (X) и predict(y), а далее проведем разбивку данных для получения тестового и тренировочного датасета   

In [ ]:
y = data['output']
X = data.drop('output', axis=1)

In [ ]:
scaler = MinMaxScaler()

X = scaler.fit_transform(X)

In [ ]:
pd.DataFrame(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.73)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

log_acc = model.score(X_test, y_test)
log_f1 = f1_score(y_test, y_pred)

In [ ]:
print(f'Логистическая регрессия:\nAccuracy:{log_acc}\nF1 Score:{log_f1}')

In [ ]:
print(f'Коэффициент детерминации для тренировочных данных: {model.score(X_train, y_train):.2%}')
print(f'Коэффициент детерминации для тестовых данных: {model.score(X_test, y_test):.2%}')

print(f'Intercept: {regr.intercept_:.2%}')

Произведем оценку наших параметров на мультиколлинеарность и определим p-value (для того чтобы понять какие из предоставленных параметров имеют статистическую значимость, а какие нет)

In [ ]:
X_incl_const = sm.add_constant(X_train)

model = sm.OLS(y_train, X_incl_const)
results = model.fit()

pd.DataFrame({'coeff': results.params, 'p-value': round(results.pvalues,3)})

Высокие значения p-value наблюдаются у следующих параметров:
* age
* chol
* fbs
* restecg
* oldpeak

Исходя из этого, можно сделать вывод, что данные по указанным параметрам не обладают высокой статистической значимостью, следовательно этими данными можно принебречь.

Для определения мультиколлинеарности воспользуемся вычислением [коэффициента увеличения дисперсии (VIF)](https://etnowiki.ru/wiki/Variance_inflation_factor)

In [ ]:
variance_inflation_factor(exog=X_incl_const.values, exog_idx=1)

In [ ]:
vif = [variance_inflation_factor(exog=X_incl_const.values, exog_idx=i) for i in range(X_incl_const.shape[1])]

pd.DataFrame({'coeff_name': X_incl_const.columns, 'vif': np.around(vif, 2)})

Так как по всем показателям в датасете мы имеем VIF < 5, то можно сделать вывод о том, что значимой мультиколлинеарности между исследуемыми параметрами нет